In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Quadratic Regression on Liquidity Index\n",
    "This notebook reproduces the regression results described in the paper *Agentic Liquidity: Economic Modeling of Interoperability and Systemic Risk in AI Ecosystems*.\n",
    "\n",
    "It loads the synthetic repo–month panel, computes the quadratic regression, and identifies the turning point L*."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "import numpy as np\n",
    "import statsmodels.formula.api as smf\n",
    "import matplotlib.pyplot as plt"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Load Panel Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {},
   "outputs": [],
   "source": [
    "df = pd.read_csv('../data/sample_repo_panel.csv')\n",
    "df['Lg2'] = df['Lg'] ** 2\n",
    "df.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Run Quadratic Regression"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [],
   "source": [
    "model = smf.ols(\n",
    "    formula='contributor_growth ~ Lg + Lg2 + org_flag + age_months',\n",
    "    data=df,\n",
    ")\n",
    "res = model.fit()\n",
    "res.summary()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Extract Turning Point L*"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "metadata": {},
   "outputs": [],
   "source": [
    "b1 = res.params['Lg']\n",
    "b2 = res.params['Lg2']\n",
    "L_star = -b1 / (2 * b2)\n",
    "print(f\"Turning point L*: {L_star:.3f}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Plot Liquidity vs Contributor Growth\n",
    "This plot generates the inverted-U shape from the synthetic panel."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "metadata": {},
   "outputs": [],
   "source": [
    "x = np.linspace(df['Lg'].min(), df['Lg'].max(), 200)\n",
    "y = res.params['Intercept'] + res.params['Lg'] * x + res.params['Lg2'] * (x ** 2)\n",
    "\n",
    "plt.figure(figsize=(8, 5))\n",
    "plt.scatter(df['Lg'], df['contributor_growth'], alpha=0.4, label='Observed')\n",
    "plt.plot(x, y, color='red', label='Quadratic fit')\n",
    "plt.axvline(L_star, color='green', linestyle='--', label=f'L* = {L_star:.2f}')\n",
    "\n",
    "plt.xlabel('Liquidity (Lg)')\n",
    "plt.ylabel('Contributor Growth')\n",
    "plt.title('Liquidity–Growth Curve (Inverted-U)')\n",
    "plt.legend()\n",
    "plt.grid(True)\n",
    "plt.show()"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "name": "python",
   "version": "3.10"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}